In [1]:
import random
import numpy as np
import torch
import os
import pathlib
import pickle
from gurobipy import *
from rsome import ro
from rsome import grb_solver as grb
import rsome as rso
from rsome import cpt_solver as cpt
import pandas as pd

In [2]:
# data_generation_process = "SPO_Data_Generation"
data_generation_process = "DDR_Data_Generation"

In [4]:
num_train = 100 # number of training data
num_feat = 4 # size of feature
num_test = 10000
deg = 1.0 # polynomial degree
e = 1.0 # scale of normal std or the range of uniform. For the error term
num_arcs = 10

lower = 0 # coef lower bound
upper = 1 # coef upper bound
p = num_feat # num of features
d = num_arcs # num of arcs
alpha = e # scale of normal std or the range of uniform. For the error term
mis = deg # model misspecification
coef_seed = 1

x_dist = 'uniform'
e_dist = 'normal'
x_low = -2
x_up = 2
x_mean = 2
x_var = 2
bump = 7
Oracle_mode = "Para"
Oracel_mode = "Realization"

In [5]:
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
grandparent_directory = os.path.dirname(parent_directory)
DataPath = os.path.dirname(grandparent_directory) + '/Data/DDR_Problem_Reproduce' + data_generation_process + "/"
pathlib.Path(DataPath).mkdir(parents=True, exist_ok=True)
print("DataPath:", DataPath)
DataPath = DataPath + "data_size="+str(num_train)+"_deg="+str(deg)+"_e="+str(e)+"_d="+str(d)+"_x_dist="+x_dist+"_diff_W/"
pathlib.Path(DataPath).mkdir(parents=True, exist_ok=True)
print(DataPath)

DataPath: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data/DDR_Problem_ReproduceDDR_Data_Generation/
/Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data/DDR_Problem_ReproduceDDR_Data_Generation/data_size=100_deg=1.0_e=1.0_d=10_x_dist=uniform_diff_W/


## Function to generate data

In [6]:
def Prepare_Data(DataPath,lower, upper, p, d, coef_seed,iteration_all,num_test, num_train, alpha,mis,data_generation_process,x_dist, e_dist, x_low, x_up, x_mean, x_var, bump):
# #  ****** Coef generation *********
    from Data import data_generation
    data_gen = data_generation()
    # W_star = data_gen.generate_truth(DataPath,lower, upper, p, d, coef_seed,data_generation_process) 
    # print("W_star = ",W_star[0,:])
    np.random.seed(coef_seed)
    x_test_all = {}; c_test_all = {}; x_train_all = {}; c_train_all = {}; W_star_all = {}; noise_train_all = {}; noise_test_all = {}
    for iter in iteration_all:
        DataPath_iter = DataPath +"iter="+str(iter)+"/"
        pathlib.Path(DataPath_iter).mkdir(parents=True, exist_ok=True)
        W_star = data_gen.generate_truth(DataPath_iter,lower, upper, p, d, iter,data_generation_process) 
        # #  ****** Data generation *********
        x_test_all[iter], c_test_all[iter], x_train_all[iter], c_train_all[iter], noise_train_all[iter],noise_test_all[iter],W_star_all[iter] = data_gen.generate_samples(iter,DataPath_iter,p, d, num_test, num_train, alpha, W_star, mis, num_test, 
                                data_generation_process, x_dist, e_dist, x_low, x_up, x_mean, x_var, bump) 
        # print()
    return x_test_all, c_test_all, x_train_all, c_train_all, noise_train_all,noise_test_all,W_star_all

# Oracle

In [7]:
iteration_all = np.arange(0,100)
x_test_all, c_test_all, x_train_all, c_train_all,noise_train_all,noise_test_all,W_star_all = Prepare_Data(DataPath,lower, upper, p, d, coef_seed,iteration_all,num_test, num_train, alpha,mis,data_generation_process,x_dist, e_dist, x_low, x_up, x_mean, x_var, bump)

In [9]:
from Peformance import DDR_Problem_Evaluation
ddr_eva = DDR_Problem_Evaluation()
cost_Oracle_para_all = {}; cost_Oracle_para_avg = {}
cost_Oracle_realization_all = {}; cost_Oracle_realization_avg = {}

for iter in iteration_all:
    # if data_generation_process == "SPO_Data_Generation":
    #     cost_oracle_ori = (W_star_all[iter] @ x_test_all[iter].T)/np.sqrt(num_feat) + 3
    #     cost_oracle_pred = (cost_oracle_ori ** mis + 1).T
    #     cost_Oracle_para_all[iter] = ddr_eva.compute_SPO_out_of_sample_Cost(cost_oracle_pred,cost_oracle_pred,noise_test_all[iter])

    if data_generation_process == "DDR_Data_Generation":
        cost_oracle_ori = (W_star_all[iter] @ x_test_all[iter].T) + bump
        cost_oracle_pred = (cost_oracle_ori ** mis).T
        cost_Oracle_para_all[iter] = ddr_eva.compute_DDR_out_of_sample_Cost(cost_oracle_pred,cost_oracle_pred,noise_test_all[iter])
        # cost_Oracle_para_avg[iter] = np.nanmean(cost_Oracle_para_all[iter])
    print("Oracle: iter=",iter,",cost_avg=",np.nanmean(cost_Oracle_para_all[iter]))

with open(DataPath+'cost_Oracle_para_all.pkl', "wb") as tf:
    pickle.dump(cost_Oracle_para_all,tf)

Oracle: iter= 0 ,cost_avg= 6.020675759557937
Oracle: iter= 1 ,cost_avg= 6.082119110674776
Oracle: iter= 2 ,cost_avg= 6.141195054673511
Oracle: iter= 3 ,cost_avg= 6.116857789943267
Oracle: iter= 4 ,cost_avg= 5.848656934594192
Oracle: iter= 5 ,cost_avg= 6.018852979876299
Oracle: iter= 6 ,cost_avg= 6.0605043896918
Oracle: iter= 7 ,cost_avg= 5.997216877224073
Oracle: iter= 8 ,cost_avg= 6.017622787443516
Oracle: iter= 9 ,cost_avg= 5.893450097073943
Oracle: iter= 10 ,cost_avg= 6.019892303726487
Oracle: iter= 11 ,cost_avg= 5.957478916252264
Oracle: iter= 12 ,cost_avg= 5.912480264404069
Oracle: iter= 13 ,cost_avg= 5.909616307013696
Oracle: iter= 14 ,cost_avg= 6.122398495155313
Oracle: iter= 15 ,cost_avg= 5.998465936889457
Oracle: iter= 16 ,cost_avg= 6.080678110877816
Oracle: iter= 17 ,cost_avg= 5.980426238151057
Oracle: iter= 18 ,cost_avg= 5.963163985216253
Oracle: iter= 19 ,cost_avg= 6.067729798752537
Oracle: iter= 20 ,cost_avg= 6.068397865415446
Oracle: iter= 21 ,cost_avg= 6.029685102246797


## OLS

In [10]:
from OLS import ols_method
ols_method_obj = ols_method()
W_ols_all = {}; w0_ols_all = {}; t_ols_all = {}; obj_ols_all = {}
cost_OLS_all = {}; cost_OLS_avg = np.zeros(len(iteration_all))
for iter in iteration_all:
    W_ols_all[iter], w0_ols_all[iter], t_ols_all[iter], obj_ols_all[iter] = ols_method_obj.ols_solver("",x_train_all[iter], c_train_all[iter])
    cost_dem = (W_ols_all[iter] @ x_test_all[iter].T).T + w0_ols_all[iter]
    # if data_generation_process == "SPO_Data_Generation":
    #     cost_oracle_ori = (W_star_all[iter] @ x_test_all[iter].T)/np.sqrt(num_feat) + 3
    #     cost_oracle_pred = (cost_oracle_ori ** mis + 1).T
    #     cost_OLS_all[iter] = perfs.compute_SPO_out_of_sample_Cost(arcs, grid,cost_dem,cost_oracle_pred,noise_test_all[iter])

    if data_generation_process == "DDR_Data_Generation":
        cost_oracle_ori = (W_star_all[iter] @ x_test_all[iter].T) + bump
        cost_oracle_pred = (cost_oracle_ori ** mis).T
        cost_OLS_all[iter] = ddr_eva.compute_DDR_out_of_sample_Cost(cost_dem,cost_oracle_pred,noise_test_all[iter])

    print("OLS: iter=",iter,",cost_avg ratio =",np.nanmean(cost_OLS_all[iter])/np.nanmean(cost_Oracle_para_all[iter]))
with open(DataPath+'cost_OLS_all.pkl', "wb") as tf:
    pickle.dump(cost_OLS_all,tf)

Set parameter Username
Academic license - for non-commercial use only - expires 2026-03-13
OLS: iter= 0 ,cost_avg ratio = 1.0063596555596261
OLS: iter= 1 ,cost_avg ratio = 1.00463993649476
OLS: iter= 2 ,cost_avg ratio = 1.008583763258777
OLS: iter= 3 ,cost_avg ratio = 1.009706807430325
OLS: iter= 4 ,cost_avg ratio = 1.0087006328675407
OLS: iter= 5 ,cost_avg ratio = 1.0039743049115024
OLS: iter= 6 ,cost_avg ratio = 1.0139353582572779
OLS: iter= 7 ,cost_avg ratio = 1.0067884486654934
OLS: iter= 8 ,cost_avg ratio = 1.0103961587898647
OLS: iter= 9 ,cost_avg ratio = 1.0127007332998774
OLS: iter= 10 ,cost_avg ratio = 1.0037415266133936
OLS: iter= 11 ,cost_avg ratio = 1.0104728628804542
OLS: iter= 12 ,cost_avg ratio = 1.0057880815322917
OLS: iter= 13 ,cost_avg ratio = 1.0104565012687508
OLS: iter= 14 ,cost_avg ratio = 1.0186072808728328
OLS: iter= 15 ,cost_avg ratio = 1.0038094784104041
OLS: iter= 16 ,cost_avg ratio = 1.0074606671341375
OLS: iter= 17 ,cost_avg ratio = 1.0076950539269707
OLS: 

# Prediction error

## DDR

In [ ]:
def solve_DDR(mu,lamb,num_acrs,num_feat,num_train,c_input,x_input):
    model = ro.Model()            # create an RSOME model
    w0_d = model.dvar(num_acrs)
    W_d = model.dvar((num_acrs,num_feat))

    alpha = model.dvar(num_train)                   

    eps = model.dvar(num_train*num_acrs)                   
    eps_index = 0
    for i in range(num_train):
        for j in range(num_acrs):
            model.st(alpha[i] >= -mu * c_input[i,j] - (1 - mu) * (W_d[j,:] @ x_input[i,:] + w0_d[j]))
            model.st(eps[eps_index] == c_input[i,j] - W_d[j,:] @ x_input[i,:] - w0_d[j])
            eps_index = eps_index + 1
    model.min(rso.norm(eps) + sum(alpha) * (lamb/num_train)) 

    model.solve(cpt,display=False,log=False)
    # print("status=",model.solution.status)
    w0_d_sol = w0_d.get()
    W_d_sol = W_d.get()
    obj = model.obj()
    return w0_d_sol,W_d_sol,obj

In [ ]:
def Loss(x_train, z_train, W, w0):
    N,p = x_train.shape
    N,d = z_train.shape
    a = []
    for n in range(N): # Number of samples
        for i in range(d): # Number of arcs
            temp = []
            for j in range(p): # Number of feature
                temp.append(x_train[n][j]*W[i,j])
            a.append((z_train[n][i] - sum(temp) - w0[i])*(z_train[n][i] - sum(temp) - w0[i]))      
    return np.sum(a)/N

In [ ]:
def ddr_solver(x_train, z_train, thres, mu, lamb):
#     x_train = data[3]
    z_train_min = np.minimum(z_train,thres) 
    N,p = x_train.shape
    N,d = z_train.shape

    # DDR
    m = Model("ddr")
    #m.setParam("DualReductions",0)
    m.setParam('OutputFlag', 0)

    W_ind = tuplelist( [(i,j) for i in range(d) for j in range(p)] )
    w0_ind = tuplelist( [i for i in range(d)] )
    t_ind = tuplelist( [n for n in range(N)] )
    
    W_ddr = m.addVars(W_ind, lb=-GRB.INFINITY )
    w0_ddr = m.addVars(w0_ind, lb=-GRB.INFINITY )
    t_ddr = m.addVars( t_ind, lb=-GRB.INFINITY )

    m.setObjective( Loss(x_train, z_train, W_ddr, w0_ddr) + lamb*(quicksum([t_ddr[n]  for n in range(N)])/ N) , GRB.MINIMIZE)

    for n in range(N):
        for i in range(d):
            m.addConstr( (- mu*z_train[n,i] - (1- mu)*(quicksum([x_train[n][j]*W_ddr[i,j] for j in range(p)]) + w0_ddr[i])-\
                    t_ddr[n] <= 0))
    m.optimize()
    W = m.getAttr('x', W_ddr)
    w0 = m.getAttr('x', w0_ddr)
    t = m.getAttr('x', t_ddr)
    W_results = []
    for i in range(d):
        W_results.append([W[(i,j)] for j in range(p)])
    w0_results = [w0[i] for i in range(d)]
#     t_results = [t[n] for n in range(N)]
    return W_results, w0_results

In [ ]:
mu_all = np.round(np.arange(0.0,1.0,0.1),4)
lamb_all = np.round(np.arange(0.0,1.0,0.1),4)

W_ddr_all = {}; w0_ddr_all = {}; obj_ddr_all = {}
cost_DDR_all = {}; cost_DDR_avg_all = {}
for iter in iteration_all:
    c_input = c_train_all[iter]
    x_input = x_train_all[iter]
    
    cost_ddr_avg_tem = np.zeros((len(mu_all),len(lamb_all)))
    for mu_index in range(len(mu_all)):
        mu = mu_all[mu_index]
        for lamb_index in range(len(lamb_all)):
            lamb = lamb_all[lamb_index]
            # print("iter = ",iter,",mu = ",mu,",lamb=",lamb)
            # w0_ddr_all[iter,mu,lamb],W_ddr_all[iter,mu,lamb],obj_ddr_all[iter,mu,lamb] = solve_DDR(mu,lamb,d,num_feat,num_train,c_input,x_input)
            W_ddr_all[iter,mu,lamb], w0_ddr_all[iter,mu,lamb] = ddr_solver(x_input, c_input, num_test, mu, lamb)
            cost_dem = (W_ddr_all[iter,mu,lamb] @ x_test_all[iter].T).T + w0_ddr_all[iter,mu,lamb]
            # if data_generation_process == "SPO_Data_Generation":
            #     cost_oracle_ori = (W_star @ x_test.T)/np.sqrt(num_feat) + 3
            #     cost_oracle_pred = (cost_oracle_ori ** mis + 1).T
            #     cost_DDR[mu,lamb] = perfs.compute_SPO_out_of_sample_Cost(arcs, grid,cost_dem,cost_oracle_pred,noise_test)

            if data_generation_process == "DDR_Data_Generation":
                cost_oracle_ori = (W_star_all[iter] @ x_test_all[iter].T) + bump
                cost_oracle_pred = (cost_oracle_ori ** mis).T
                cost_DDR_all[iter,mu,lamb] = ddr_eva.compute_DDR_out_of_sample_Cost(cost_dem,cost_oracle_pred,noise_test_all[iter])

    print("iter = ",iter,"======")

### Average performance

In [ ]:
cost_Oracle_avg_all = {}; cost_OLS_avg_all = {}; cost_DDR_avg_all = {}
regret_DDR_vs_OLS_all = {}; regret_DDR_vs_OLS_avg_all = np.zeros((len(mu_all),len(lamb_all)))
cost_reduction_DDR_vs_OLS_all = {}; cost_reduction_DDR_vs_OLS_avg_all = np.zeros((len(mu_all),len(lamb_all)))

for iter in iteration_all:
    cost_Oracle_avg_all[iter] = np.nanmean(cost_Oracle_para_all[iter])
    cost_OLS_avg_all[iter] = np.nanmean(cost_OLS_all[iter])

    cost_ddr_avg_mat = np.zeros((len(mu_all),len(lamb_all)))
    for mu_index in range(len(mu_all)):
        mu = mu_all[mu_index]
        for lamb_index in range(len(lamb_all)):
            lamb = lamb_all[lamb_index]
            cost_ddr_avg_mat[mu_index,lamb_index] = np.nanmean(cost_DDR_all[iter,mu,lamb])

    cost_DDR_avg_all[iter] = cost_ddr_avg_mat
    regret_DDR_vs_OLS_all[iter] = (cost_OLS_avg_all[iter] - cost_ddr_avg_mat)/np.abs(cost_OLS_avg_all[iter] - cost_Oracle_avg_all[iter])
    regret_DDR_vs_OLS_avg_all = regret_DDR_vs_OLS_avg_all + regret_DDR_vs_OLS_all[iter]

    cost_reduction_DDR_vs_OLS_all[iter] = (cost_OLS_avg_all[iter] - cost_ddr_avg_mat)/np.abs(cost_OLS_avg_all[iter])
    cost_reduction_DDR_vs_OLS_avg_all = cost_reduction_DDR_vs_OLS_avg_all + cost_reduction_DDR_vs_OLS_all[iter]

    print("iter = ",iter,"======")
regret_DDR_vs_OLS_avg_all = regret_DDR_vs_OLS_avg_all/len(iteration_all)
cost_reduction_DDR_vs_OLS_avg_all = cost_reduction_DDR_vs_OLS_avg_all/len(iteration_all)

In [ ]:
regret_DDR_vs_OLS_avg_df = pd.DataFrame(regret_DDR_vs_OLS_avg_all)
regret_DDR_vs_OLS_avg_df.index = ["$\mu="+str(mu)+"$" for mu in mu_all]
regret_DDR_vs_OLS_avg_df.columns = ["$\lambda="+str(lamb)+"$" for lamb in lamb_all]
regret_DDR_vs_OLS_avg_df.to_csv(DataPath+"regret_DDR_vs_OLS_avg.csv")

cost_reduction_DDR_vs_OLS_avg_df = pd.DataFrame(cost_reduction_DDR_vs_OLS_avg_all)
cost_reduction_DDR_vs_OLS_avg_df.index = ["$\mu="+str(mu)+"$" for mu in mu_all]
cost_reduction_DDR_vs_OLS_avg_df.columns = ["$\lambda="+str(lamb)+"$" for lamb in lamb_all]
cost_reduction_DDR_vs_OLS_avg_df.to_csv(DataPath+"cost_reduction_DDR_vs_OLS_avg.csv")